[Using Virtual Environments in Jupyter Notebook and Python](https://janakiev.com/blog/jupyter-virtual-envs/)


[주피터 (Jupyter Notebook) 설치하여 웹브라우저로 서버 관리 - 우분투](https://hithot.tistory.com/entry/%EC%A3%BC%ED%94%BC%ED%84%B0-Jupyter-Notebook-%EC%84%A4%EC%B9%98%ED%95%98%EC%97%AC-%EC%9B%B9%EB%B8%8C%EB%9D%BC%EC%9A%B0%EC%A0%80%EB%A1%9C-%EC%84%9C%EB%B2%84-%EA%B4%80%EB%A6%AC-%EC%9A%B0%EB%B6%84%ED%88%AC?category=823577)

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

print(torch.rand(5,3))
print(torch.cuda.is_available())

tensor([[0.8888, 0.3437, 0.0741],
        [0.2750, 0.4212, 0.5925],
        [0.1469, 0.9967, 0.2159],
        [0.2730, 0.4382, 0.2797],
        [0.1739, 0.6737, 0.2798]])
False


In [2]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# each X, y is a batch of {batch_size} datapoints
cnt = 0
for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    cnt += 1
    if cnt > 5:
        break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64
Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64
Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64
Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64
Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64
Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [9]:
# 학습에 사용할 CPU나 GPU 장치를 얻습니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [19]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [20]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
%%capture output
# Save, close tab, come back later. The output is now stored in the output variable:
# output.show()
# This will show all interim print results as well as the plain or rich output cell.

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

In [33]:
output.show()

Epoch 1
-------------------------------
loss: 1.846454  [    0/60000]
loss: 1.840754  [ 6400/60000]
loss: 1.688818  [12800/60000]
loss: 1.605958  [19200/60000]
loss: 1.861017  [25600/60000]
loss: 1.867185  [32000/60000]
loss: 1.773537  [38400/60000]
loss: 1.907231  [44800/60000]
loss: 1.729227  [51200/60000]
loss: 1.645265  [57600/60000]
Test Error: 
 Accuracy: 46.8%, Avg loss: 0.027309 

Epoch 2
-------------------------------
loss: 1.846454  [    0/60000]
loss: 1.840754  [ 6400/60000]
loss: 1.688818  [12800/60000]
loss: 1.605958  [19200/60000]
loss: 1.861017  [25600/60000]
loss: 1.867185  [32000/60000]
loss: 1.773537  [38400/60000]
loss: 1.907231  [44800/60000]
loss: 1.729227  [51200/60000]
loss: 1.645265  [57600/60000]
Test Error: 
 Accuracy: 46.8%, Avg loss: 0.027309 

Epoch 3
-------------------------------
loss: 1.846454  [    0/60000]
loss: 1.840754  [ 6400/60000]
loss: 1.688818  [12800/60000]
loss: 1.605958  [19200/60000]
loss: 1.861017  [25600/60000]
loss: 1.867185  [32000/600

## 모델 저장 & 불러오기

In [26]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [27]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [29]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Sneaker", Actual: "Ankle boot"


In [24]:
3+3

6

In [10]:
import torch

In [11]:
dir(torch)

['AVG',
 'AggregationType',
 'AnyType',
 'Argument',
 'ArgumentSpec',
 'BFloat16Storage',
 'BFloat16Tensor',
 'BenchmarkConfig',
 'BenchmarkExecutionStats',
 'Block',
 'BoolStorage',
 'BoolTensor',
 'BoolType',
 'BufferDict',
 'ByteStorage',
 'ByteTensor',
 'CONV_BN_FUSION',
 'CallStack',
 'Capsule',
 'CharStorage',
 'CharTensor',
 'ClassType',
 'Code',
 'CompilationUnit',
 'CompleteArgumentSpec',
 'ComplexDoubleStorage',
 'ComplexFloatStorage',
 'ComplexType',
 'ConcreteModuleType',
 'ConcreteModuleTypeBuilder',
 'DeepCopyMemoTable',
 'DeviceObjType',
 'DictType',
 'DisableTorchFunction',
 'DoubleStorage',
 'DoubleTensor',
 'EnumType',
 'ErrorReport',
 'ExecutionPlan',
 'FUSE_ADD_RELU',
 'FatalError',
 'FileCheck',
 'FloatStorage',
 'FloatTensor',
 'FloatType',
 'FunctionSchema',
 'Future',
 'FutureType',
 'Generator',
 'Gradient',
 'Graph',
 'GraphExecutorState',
 'HOIST_CONV_PACKED_PARAMS',
 'HalfStorage',
 'HalfStorageBase',
 'HalfTensor',
 'INSERT_FOLD_PREPACK_OPS',
 'IODescriptor

In [16]:
m = nn.Linear(5, 6)
input = torch.randn(10, 5)
output = m(input)
print(output.size())

torch.Size([10, 6])


In [17]:
input, output

(tensor([[ 1.9962,  0.0996,  0.1479, -1.0129, -0.0659],
         [-0.2024,  0.9311,  0.5858, -0.5480,  0.6899],
         [ 1.1325,  1.1787, -0.5252,  0.4131, -0.1455],
         [-0.5413,  1.9641, -1.2753, -0.0184,  1.5657],
         [ 0.5589,  0.0896,  0.0702,  0.0064, -0.6095],
         [ 0.4696, -2.1550,  0.2622,  0.6374, -0.6430],
         [ 1.4073, -0.4629, -0.5124, -0.5942, -0.4861],
         [-0.6663, -0.5769,  0.0107, -0.7793, -2.4083],
         [-0.1826, -0.5537,  1.1015, -1.6887,  0.2530],
         [ 0.1800, -0.0770,  0.5635,  0.5061, -1.5352]]),
 tensor([[-0.8794,  0.6896, -0.5001,  0.3345,  0.0324, -0.4620],
         [-0.1005,  0.0320, -0.8660, -0.3718, -0.2444,  0.6927],
         [ 0.2557,  0.5111, -0.6527, -0.5666, -0.5598, -0.0506],
         [ 0.7434, -0.8242, -0.7764, -1.7511, -1.1434,  1.4386],
         [ 0.0441,  0.4220, -0.4058,  0.0872,  0.1689, -0.4998],
         [ 0.0397,  0.5646,  0.3889,  1.0292,  0.6375, -1.4956],
         [-0.3732,  0.3120, -0.1057,  0.2360,  0

In [32]:
output.show()

Epoch 1
-------------------------------
loss: 1.846454  [    0/60000]
loss: 1.840754  [ 6400/60000]
loss: 1.688818  [12800/60000]
loss: 1.605958  [19200/60000]
loss: 1.861017  [25600/60000]
loss: 1.867185  [32000/60000]
loss: 1.773537  [38400/60000]
loss: 1.907231  [44800/60000]
loss: 1.729227  [51200/60000]
loss: 1.645265  [57600/60000]
Test Error: 
 Accuracy: 46.8%, Avg loss: 0.027309 

Epoch 2
-------------------------------
loss: 1.846454  [    0/60000]
loss: 1.840754  [ 6400/60000]
loss: 1.688818  [12800/60000]
loss: 1.605958  [19200/60000]
loss: 1.861017  [25600/60000]
loss: 1.867185  [32000/60000]
loss: 1.773537  [38400/60000]
loss: 1.907231  [44800/60000]
loss: 1.729227  [51200/60000]
loss: 1.645265  [57600/60000]
Test Error: 
 Accuracy: 46.8%, Avg loss: 0.027309 

Epoch 3
-------------------------------
loss: 1.846454  [    0/60000]
loss: 1.840754  [ 6400/60000]
loss: 1.688818  [12800/60000]
loss: 1.605958  [19200/60000]
loss: 1.861017  [25600/60000]
loss: 1.867185  [32000/600